In [1]:
import numpy as np
import glob, os
from keras import layers as ly
from keras.models import Sequential, load_model, Model
from keras.layers import LSTM, Dense, RepeatVector, TimeDistributed
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.layers import Input, Lambda
from keras import losses
from keras.models import model_from_json
import keras.backend.tensorflow_backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
import string

Using TensorFlow backend.


KeyboardInterrupt: 

In [13]:
#dir = './sequence/*'
dir = './latest_sequence/bfs-character/*'

In [41]:
# file read
name = []
all_data = []
sequence_length = []
alpha = list(string.ascii_uppercase)
for file in sorted(glob.glob(dir)):
    name.append(file.split('/')[-1].replace('.txt', ''))
    datasets = []
    for f in open(file, 'r'):
        f = f.replace(']', '').replace('[', '').replace('\n','')
        (u, v, w) = f.split(', ')
        datasets.append([alpha.index(u[1])+1, alpha.index(v[1]) +1, float(w)])
    sequence_length.append(len(datasets))
    all_data.append(datasets)
#all_data = np.array(all_data)
all_data = np.array([np.array(arr) for arr in all_data])

In [42]:
x_train, x_test, train_name, test_name = train_test_split(all_data, name, test_size=0.3)
x_test, x_val, test_name, val_name = train_test_split(x_test, test_name, test_size=0.33)

In [43]:
train_name
tr_names= []
for name in train_name:
    tr_names.append(name.split('-')[0].replace('graph', ''))

In [6]:
max_sequence_length = max(sequence_length)
n_features = 3
batch_size = 32
epochs = 500
steps_per_epoch = len(x_train)

In [7]:
def custom_loss(y_true, y_pred):
    loss1 = losses.mean_squared_error(y_true, y_pred)
    loss2 = losses.kld(y_true, y_pred) # categorical_crossentropy
    return loss1 * 0.7 + loss2 * 0.3

In [8]:
def repeat_vector(args):
    layer_to_repeat = args[0]
    sequence_layer = args[1]
    return RepeatVector(K.shape(sequence_layer)[1])(layer_to_repeat)
    

inputs = Input(shape=(None, 3))
encoded = LSTM(128, return_sequences=True)(inputs)  #activation 안적으면 tanh
encoded = LSTM(64)(encoded)

decoded = Lambda(repeat_vector, output_shape=(None, 64)) ([encoded, inputs]) # inputs의 shape[1] 만큼 encoded 를 반복 생성

decoded = LSTM(64, return_sequences=True)(decoded)
decoded = LSTM(128, return_sequences=True)(decoded)
decoded = TimeDistributed(Dense(3))(decoded)
encoder = Model(inputs, encoded)

lstm_autoencoder = Model(inputs, decoded)
lstm_autoencoder.compile(loss=losses.mean_squared_error, optimizer='adam')
#lstm_autoencoder_500 = lstm_autoencoder

W1007 23:52:07.814294 139656679192320 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1007 23:52:07.837179 139656679192320 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1007 23:52:07.843047 139656679192320 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1007 23:52:09.118702 139656679192320 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [9]:
def train_generator(x_train):
    idx = 0
    while True:
        yield np.array([x_train[idx]]), np.array([x_train[idx]])
        idx +=1
        if idx >= len(x_train):
            idx = 0

In [10]:
model_path = 'model_save/mse_kld_models/weights' + '{epoch:02d}-{loss:.4f}.h5'
early_stopping_callback = EarlyStopping(monitor='loss', patience=200)
checkpoint_callback = ModelCheckpoint(model_path, monitor='loss', verbose = 1, save_best_only = True, save_weights_only = True, mode='min')#, period=5)

In [12]:
lstm_autoencoder.fit_generator(train_generator(x_train), epochs=500, steps_per_epoch=steps_per_epoch, verbose=1, callbacks=[early_stopping_callback, checkpoint_callback])

Epoch 1/500
2360/2360 [==============================] - 77s 33ms/step - loss: 38.8765

Epoch 00001: loss improved from inf to 38.87647, saving model to result/mse_kld_models/weights01-38.8765.h5
Epoch 2/500
2360/2360 [==============================] - 59s 25ms/step - loss: 38.5474

Epoch 00002: loss improved from 38.87647 to 38.54742, saving model to result/mse_kld_models/weights02-38.5474.h5
Epoch 3/500
2360/2360 [==============================] - 46s 19ms/step - loss: 38.5902

Epoch 00003: loss did not improve from 38.54742
Epoch 4/500
2360/2360 [==============================] - 38s 16ms/step - loss: 38.5874

Epoch 00004: loss did not improve from 38.54742
Epoch 5/500
2360/2360 [==============================] - 49s 21ms/step - loss: 26.7460

Epoch 00005: loss improved from 38.54742 to 26.74596, saving model to result/mse_kld_models/weights05-26.7460.h5
Epoch 6/500
2360/2360 [==============================] - 46s 20ms/step - loss: 15.5021

Epoch 00006: loss improved from 26.74596 t

2360/2360 [==============================] - 36s 15ms/step - loss: 0.5168

Epoch 00098: loss improved from 0.56125 to 0.51679, saving model to result/mse_kld_models/weights98-0.5168.h5
Epoch 99/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.4507

Epoch 00099: loss improved from 0.51679 to 0.45072, saving model to result/mse_kld_models/weights99-0.4507.h5
Epoch 100/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.4585

Epoch 00100: loss did not improve from 0.45072
Epoch 101/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.4400

Epoch 00101: loss improved from 0.45072 to 0.43997, saving model to result/mse_kld_models/weights101-0.4400.h5
Epoch 102/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.4601

Epoch 00102: loss did not improve from 0.43997
Epoch 103/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.5720

Epoch 00103: loss did not improve from 0.43997
Epoch 104

2360/2360 [==============================] - 36s 15ms/step - loss: 0.2575

Epoch 00151: loss did not improve from 0.21353
Epoch 152/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.2144

Epoch 00152: loss did not improve from 0.21353
Epoch 153/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.2122

Epoch 00153: loss improved from 0.21353 to 0.21225, saving model to result/mse_kld_models/weights153-0.2122.h5
Epoch 154/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.2173

Epoch 00154: loss did not improve from 0.21225
Epoch 155/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.2104

Epoch 00155: loss improved from 0.21225 to 0.21040, saving model to result/mse_kld_models/weights155-0.2104.h5
Epoch 156/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.2027

Epoch 00156: loss improved from 0.21040 to 0.20270, saving model to result/mse_kld_models/weights156-0.2027.h5
Epoch 

2360/2360 [==============================] - 36s 15ms/step - loss: 0.1227

Epoch 00264: loss did not improve from 0.11336
Epoch 265/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.1207

Epoch 00265: loss did not improve from 0.11336
Epoch 266/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.1235

Epoch 00266: loss did not improve from 0.11336
Epoch 267/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.1175

Epoch 00267: loss did not improve from 0.11336
Epoch 268/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.1193

Epoch 00268: loss did not improve from 0.11336
Epoch 269/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.1243

Epoch 00269: loss did not improve from 0.11336
Epoch 270/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.1220

Epoch 00270: loss did not improve from 0.11336
Epoch 271/500
2360/2360 [==============================] - 36s

2360/2360 [==============================] - 36s 15ms/step - loss: 0.1020

Epoch 00322: loss did not improve from 0.09699
Epoch 323/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.1089

Epoch 00323: loss did not improve from 0.09699
Epoch 324/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.1163

Epoch 00324: loss did not improve from 0.09699
Epoch 325/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.1053

Epoch 00325: loss did not improve from 0.09699
Epoch 326/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.1175

Epoch 00326: loss did not improve from 0.09699
Epoch 327/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.0943

Epoch 00327: loss improved from 0.09699 to 0.09427, saving model to result/mse_kld_models/weights327-0.0943.h5
Epoch 328/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.1161

Epoch 00328: loss did not improve from 0.0942

2360/2360 [==============================] - 36s 15ms/step - loss: 0.0902

Epoch 00381: loss did not improve from 0.08639
Epoch 382/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.1119

Epoch 00382: loss did not improve from 0.08639
Epoch 383/500
2360/2360 [==============================] - 35s 15ms/step - loss: 0.0929

Epoch 00383: loss did not improve from 0.08639
Epoch 384/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.3202

Epoch 00384: loss did not improve from 0.08639
Epoch 385/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.1699

Epoch 00385: loss did not improve from 0.08639
Epoch 386/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.0817

Epoch 00386: loss improved from 0.08639 to 0.08167, saving model to result/mse_kld_models/weights386-0.0817.h5
Epoch 387/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.0880

Epoch 00387: loss did not improve from 0.0816

2360/2360 [==============================] - 36s 15ms/step - loss: 0.0878

Epoch 00440: loss did not improve from 0.07898
Epoch 441/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.0961

Epoch 00441: loss did not improve from 0.07898
Epoch 442/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.0849

Epoch 00442: loss did not improve from 0.07898
Epoch 443/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.0776

Epoch 00443: loss improved from 0.07898 to 0.07760, saving model to result/mse_kld_models/weights443-0.0776.h5
Epoch 444/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.0876

Epoch 00444: loss did not improve from 0.07760
Epoch 445/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.1850

Epoch 00445: loss did not improve from 0.07760
Epoch 446/500
2360/2360 [==============================] - 36s 15ms/step - loss: 0.1243

Epoch 00446: loss did not improve from 0.0776

2360/2360 [==============================] - 19s 8ms/step - loss: 0.0701

Epoch 00499: loss did not improve from 0.06472
Epoch 500/500
2360/2360 [==============================] - 19s 8ms/step - loss: 0.0783

Epoch 00500: loss did not improve from 0.06472


In [13]:
# MODEL SAVE
model_json = lstm_autoencoder.to_json()
filename = 'latest_mse_kld_lstmae' #input('filename: ') 
with open('model_save/mse_kld_models/' + filename + '.json', 'w') as file:
    file.write(model_json)
lstm_autoencoder.save_weights('model_save/mse_kld_models/weights_' +  filename + '.h5')

In [7]:
filename = "latest_mse_kld_lstmae"
# MODEL LOAD
loaded_model = model_from_json(open('model_save/mse_kld_models/' +filename + '.json').read())
loaded_model.load_weights('model_save/mse_kld_models/weights_' + filename + '.h5')

W1010 15:50:46.376312 140028240148224 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1010 15:50:46.386179 140028240148224 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1010 15:50:46.388441 140028240148224 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1010 15:50:46.865248 140028240148224 deprecation_wrapper.py:119] From /home/minji/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



In [14]:
loaded_model = lstm_autoencoder

In [8]:
mean= 0
for xt in x_test:
    xt = xt.reshape(1, xt.shape[0], xt.shape[1])
    out = loaded_model.predict(xt)
    mean += ((xt-out)**2).mean(axis=None)
print(mean/len(x_test))

0.3130180908054783


In [ ]:
encoder = Model(loaded_model.input, loaded_model.layers[3].output)

In [ ]:
test = x_test[0].reshape(1, x_test[0].shape[0], x_test[0].shape[1])
latent_vector = []
for x in x_test:
    x = x.reshape(1, x.shape[0], x.shape[1])
    latent_vector.append(encoder.predict(x)[0])